In [1]:
##DATA INGESTION and DATA WRANGLING

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)
import seaborn as sns
import re
import json
import requests
import subprocess
import glob
import os

In [3]:
#!pip install geopandas
#!pip install pygeocoder
#!pip install uszipcode

In [4]:
import geopandas as gpd
from geopandas import GeoDataFrame
from pygeocoder import Geocoder

/Applications/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [5]:
# get data: Towing Report from Montgomery Data 
#towing = pd.DataFrame(requests.get('https://data.montgomerycountymd.gov/resource/i6vn-3s6e.json').json())
url = 'https://data.montgomerycountymd.gov/api/views/i6vn-3s6e/rows.csv?accessType=DOWNLOAD'
towing = pd.read_csv(url)
towing.head()



,Tow Date,Vehicle Year,Vehicle Make,Vehicle Model,Tow Reason,Storage Company,Notes,Incident Date,Location,City,Geo-location field
0,04/30/2020,2019.0,JEEP,NaN,NOP,TCL,NaN,05/02/0020,19757 CRYSTAL ROCK DR,GERMANTOWN,POINT (-77.261484 39.179888)
1,06/13/2020,2015.0,JEEP,GCH,FIRE,TCL,NaN,06/15/0020,20107 TORREY POND PL,MONTGOMERY VILLAGE,POINT (-77.204705 39.185963)
2,05/21/2020,2005.0,GMC,NaN,FIRE,tcl,FIRE LANE // HENRYS 301-927-0007 // TCL,05/23/0020,6660 EAMES WAY,BETHESDA,NaN
3,04/17/2020,2015.0,HYUN,ACC,NOP,A&M,NOP A&M 301-421-0953,04/19/0020,100 BLK BOWSPRIT CT,GAITHERSBURG,NaN
4,05/24/2020,2015.0,MINI,COOPER,NOP,tcl,NO PERMIT // AUBURN // 301-977-2740 // TCL,05/26/0020,ARIZONA CIR / WESTLAKE DR,BETHESDA,NaN


In [6]:
#Rename columns
towing.columns = ["tow_date", "vehicle_year", "vehicle_make", "vehicle_model", "tow_reason", "storage_company", "notes", "incident_date", "street", "city", "geo_location"]
        

In [7]:
#capitalize on first letter of each string in towing.notes, towing.story_company, towing.location, towing.city
towing.loc[:,"notes"] = towing.loc[:,"notes"].str.title()
towing.loc[:,"storage_company"] = towing.loc[:,"storage_company"].str.title()
towing.loc[:,"street"] = towing.loc[:,"street"].str.title()
towing.loc[:,"city"] = towing.loc[:,"city"].str.title()
towing.head()
towing.dtypes

tow_date            object
vehicle_year       float64
vehicle_make        object
vehicle_model       object
tow_reason          object
storage_company     object
notes               object
incident_date       object
street              object
city                object
geo_location        object
dtype: object

In [8]:
#drop columns not needed
towing.drop('vehicle_year', axis=1, inplace=True) 
towing.drop('vehicle_make', axis=1, inplace=True)
towing.drop('vehicle_model', axis=1, inplace=True)

In [9]:
#remove apostrophe in notes and storage company before capitalizing with str.title
towing["notes"] = towing["notes"].str.replace("'", "")   #remove apostrophe
towing["storage_company"] = towing["storage_company"].str.replace("'", "")   #remove apostrophe

#capitalize on first letter of each string in towing.notes, towing.storage_company, towing.city
towing.loc[:,"storage_company"] = towing.loc[:,"storage_company"].str.title()
towing.loc[:,"city"] = towing.loc[:,"city"].str.title()

#other string fixes
towing["tow_reason"] = towing["tow_reason"].str.upper() 
towing["storage_company"] = towing["storage_company"].str.replace("Rickys ", "Rickies") #fix mispelling
towing["notes"] = towing["notes"].str.replace("Rickys ", "Rickies")
towing["storage_company"] = towing["storage_company"].str.replace("HenryS", "Henrys") #fix mispelling
towing["notes"] = towing["notes"].str.replace("HenryS", "Henrys")
towing["notes"] = towing["notes"].str.replace("HenryS", "Henrys")
towing["notes"] = towing["notes"].str.replace("Herny'S", "Henrys")
towing["notes"] = towing["notes"].str.replace("HenrS", "Henrys")
towing["notes"] = towing["notes"].str.replace("Henrus", "Henrys")
towing["notes"] = towing["notes"].str.replace("HenrysTowing", "Henrys")
towing["notes"] = towing["notes"].str.replace("Henry;'S", "Henrys")
towing["notes"] = towing["notes"].str.replace("Henrsy", "Henrys")
towing["notes"] = towing["notes"].str.replace("Henr'S", "Henrys")
towing["notes"] = towing["notes"].str.replace("Milllers", "Millers")
towing["notes"] = towing["notes"].str.replace("MillerS", "Millers")
towing["notes"] = towing["notes"].str.replace("Prestege", "Prestige")
towing["notes"] = towing["notes"].str.replace("Pretige", "Prestige")
towing["notes"] = towing["notes"].str.replace("Pretisge", "Prestige")
towing["notes"] = towing["notes"].str.replace("Presteig", "Prestige")
towing["notes"] = towing["notes"].str.replace("Prestigee", "Prestige")
towing["notes"] = towing["notes"].str.replace("Presige", "Prestige")
towing["notes"] = towing["notes"].str.replace("Prestiage", "Prestige")
towing["notes"] = towing["notes"].str.replace("Pretisge", "Prestige")
towing["notes"] = towing["notes"].str.replace("Auhtorized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Athorized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authortized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Autherized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authorizd", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authoriaed", "Authorized")
towing["notes"] = towing["notes"].str.replace("Uthorized", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authorizerd", "Authorized")
towing["notes"] = towing["notes"].str.replace("Authorixzed", "Authorized")
towing["notes"] = towing["notes"].str.replace("Autorized", "Authorized")
towing["notes"] =  towing["notes"].str.replace("RickyS", "Rickys")
towing["notes"] = towing["notes"].str.replace("Rickeys", "Rickys")
towing["notes"] = towing["notes"].str.replace("Rckys", "Rickys")
towing["notes"] = towing["notes"].str.replace("RickS", "Rickys")
towing["notes"] = towing["notes"].str.replace("Ricky", "Rickys")
towing["notes"] = towing["notes"].str.replace("SpeedyS", "Speedys")
towing["notes"] = towing["notes"].str.replace("Diviersified", "Diversified")
towing["notes"] = towing["notes"].str.replace("Deversified", "Diversified")
towing["notes"] = towing["notes"].str.replace("Diviersified", "Diversified")
towing["notes"] = towing["notes"].str.replace("Diversied", "Diversified")

towing["storage_company"] = towing["storage_company"].str.replace("Autherized", "Authorized")
towing["storage_company"] = towing["storage_company"].str.replace("Autorized Towing", "Authorized")
towing["storage_company"] = towing["storage_company"].str.replace("HenrysTowing", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("Henry", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("Henryss", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("Henreys", "Henrys")
towing["storage_company"] = towing["storage_company"].str.replace("Henrys Towin", "Henrys")


#convert date to datetime object
towing['tow_date'] = pd.to_datetime(towing['tow_date'])



towing.head()
print(towing.count())
towing.dtypes

tow_date           20415
tow_reason         20408
storage_company    19649
notes              19963
incident_date      20424
street             20424
city               20424
geo_location       20078
dtype: int64


tow_date           datetime64[ns]
tow_reason                 object
storage_company            object
notes                      object
incident_date              object
street                     object
city                       object
geo_location               object
dtype: object

In [10]:
#eliminate rows with Nan
towing = towing.dropna()
towing = towing.reset_index(drop=True)
#change text columns to string datatype
towing["notes"].astype(str)
towing["storage_company"].astype(str)
towing["city"].astype(str)
towing["tow_reason"].astype(str)
# remove special characters from notes column
towing["notes"] = towing["notes"].str.replace("//", " ")
towing["notes"] = towing["notes"].str.replace("/", " ")
towing["notes"] = towing["notes"].str.replace(':', " ")
towing["notes"] = towing["notes"].str.replace('-', "")
towing["notes"] = towing["notes"].str.replace('#', "")
towing["notes"] = towing["notes"].str.replace(".", "")
towing["notes"] = towing["notes"].str.replace(",", "")
towing["notes"] = towing["notes"].str.replace('@', "")
towing["notes"] = towing["notes"].str.replace("Rickys", "Rickies")
towing["notes"] = towing["notes"].str.replace("Tcl", "")

# Use regex techniques on notes column
towing['notes'] = towing['notes'].str.replace(r'\d+','')
towing["notes"] = towing["notes"].str.replace('Repo ', "")
towing["notes"] = towing["notes"].str.replace('Walk Off ', "")
towing["notes"] = towing["notes"].str.replace('Nop ', "")
towing["notes"] = towing["notes"].str.replace("Nop", "")
towing["notes"] = towing["notes"].str.replace('Fire Lane ', "")
towing["notes"] = towing["notes"].str.replace('Vin ', "")
towing["notes"] = towing["notes"].str.replace('Fin. ', "")
towing["notes"] = towing["notes"].str.replace('Prv ', "")
towing["notes"] = towing["notes"].str.replace("No Permit ", "")
towing["notes"] = towing["notes"].str.replace('Unauthorized ', "")
towing["notes"] = towing["notes"].str.replace('Unautorized', "")
towing["notes"] = towing["notes"].str.replace('Parking ', "")
towing["notes"] = towing["notes"].str.replace('Pkg', "")
towing["notes"] = towing["notes"].str.replace('Management ', "")
towing["notes"] = towing["notes"].str.replace('Mgmt', "")
towing["notes"] = towing["notes"].str.replace('Mgt ', "")
towing["notes"] = towing["notes"].str.replace('Request ', "")
towing["notes"] = towing["notes"].str.replace('Req ', "")
towing["notes"] = towing["notes"].str.replace('Final ', "")
towing["notes"] = towing["notes"].str.replace('Notice ', "")
towing["notes"] = towing["notes"].str.replace('No Permit ', "")
towing["notes"] = towing["notes"].str.replace('Invalid Permit ', "")
towing["notes"] = towing["notes"].str.replace("Expired Permit ", "")
towing["notes"] = towing["notes"].str.replace("Exp Permit ", "")
towing["notes"] = towing["notes"].str.replace("Btl ", "")
towing["notes"] = towing["notes"].str.replace("Rickys ", "Rickies")
towing["notes"] = towing["notes"].str.replace("No Commercial Vehs", "")
towing["notes"] = towing["notes"].str.replace("Parked In Reserved Spot", "")
towing["notes"] = towing["notes"].str.replace("Parked In Handicap Without Permit", "")
towing["notes"] = towing["notes"].str.replace("No Overnight ", "")
towing["notes"] = towing["notes"].str.replace("Handicapped ", "")
towing["notes"] = towing["notes"].str.replace("Taking Two Spaces ", "")
towing["notes"] = towing["notes"].str.replace("Reserved Space ", "")
towing["notes"] = towing["notes"].str.replace("Marked Space ", "")
towing["notes"] = towing["notes"].str.replace("Back To The Lot ", "") 
towing["notes"] = towing["notes"].str.replace("()", "")
towing["notes"] = towing["notes"].str.replace("Displayed ", "")
towing["notes"] = towing["notes"].str.replace("Parked In", "")
towing["notes"] = towing["notes"].str.replace("Double Parked", "")
towing["notes"] = towing["notes"].str.replace("No Zone", "")
towing["notes"] = towing["notes"].str.replace("No Permit ", "")
towing["notes"] = towing["notes"].str.replace("Unreadable", "") 
towing["notes"] = towing["notes"].str.replace("Blocked", "")
towing["notes"] = towing["notes"].str.replace("Per Mgmnt", "")
towing["notes"] = towing["notes"].str.replace("Blk Access", "")
towing["notes"] = towing["notes"].str.replace("Visitor Spot Over Time Limit", "")
towing["notes"] = towing["notes"].str.replace("On Private Property", "")
towing["notes"] = towing["notes"].str.replace("Over Time Limit For Visitor", "")
towing["notes"] = towing["notes"].str.replace("Illegally Parked", "")
towing["notes"] = towing["notes"].str.replace("Expired Permit", "")
towing["notes"] = towing["notes"].str.replace("Blocking Access", "")
towing["notes"] = towing["notes"].str.replace("Covered", "")

towing.tail()

,tow_date,tow_reason,storage_company,notes,incident_date,street,city,geo_location
18859,2021-02-16,PKG,Tcl,Henrys Fire Ln,02/18/0021,3536 Gentry Ridge Ct,Silver Spring,POINT (-76.939653 39.072248)
18860,2021-02-10,NOP,Authorized,Authorized,10/23/0002,14120 Weeping Willow Dr,Silver Spring,POINT (-77.06692 39.088854)
18861,2021-02-24,PKG,Tcl,Authorized,02/26/0021,14120 Weeping Willow Dr,Silver Spring,POINT (-77.06692 39.088854)
18862,2021-01-15,EXP,Drive,Drive,01/17/0021,8658 Piney Branch Rd,Silver Spring,POINT (-76.999533 38.998597)
18863,2021-02-14,REPO,Tcl,Santander Notic,02/16/0021,8847 Thomas Lea Ter,Montgomery Village,POINT (-77.181901 39.191003)


In [11]:
#create list of company names ocurring 
company_count = towing["storage_company"].value_counts()
company_freq = towing["storage_company"].value_counts(normalize = True)
company = pd.Series(towing["storage_company"])
names = company.drop_duplicates()
top_names = names
#print(company_count)
#print(company_freq)
print(top_names)
len(towing)

0                              Henrys
1                                 Tcl
2                            Prestige
5                          Authorized
9                                 A&M
                     ...             
18100    Flowmax Towing And Transport
18322                               H
18387                   14505 Lee Hwy
18552                            Avis
18651                             Cgi
Name: storage_company, Length: 259, dtype: object


18864

In [12]:
#Finish cleaning Storage Company


word_set = set(top_names)
x=-1
rowcount = len(towing)-1
while x < rowcount :
    x = x+1
    if towing.iloc[x,2] == "Tcl" :
        string_to_be_searched = str(towing.iloc[x,3])
        for w in string_to_be_searched.split():
            if w in word_set :
                towing.iloc[x,2] = w
                print("%s in string" % w)    
            else :
                print("not found ", w)
    else :
        print(x, "is ok")
        
towing.head(50)
print(tow["storage_company" =="Tcl"], tow["notes"])



       

0 is ok
Authorized in string
2 is ok
Henrys in string
Henrys in string
5 is ok
Prestige in string
7 is ok
Henrys in string
9 is ok
10 is ok
not found  Illegally
not found  Pkd
not found  Pkd
not found  Overnight
Custom in string
Henrys in string
Henrys in string
14 is ok
Henrys in string
Authorize in string
Prestigue in string
not found  Towing
Authorized in string
not found  Towing
not found  To
not found  For
not found  No
not found  Zone
Authorized in string
20 is ok
21 is ok
22 is ok
23 is ok
24 is ok
25 is ok
26 is ok
not found  Handicap
not found  W
not found  O
not found  Tag
not found  Or
not found  Permit
Authorized in string
Led in string
not found  Towing
29 is ok
Speedy in string
Henrys in string
not found  Towing
not found  Btl
Auth in string
not found  Handicap
33 is ok
Authorized in string
Authorized in string
Prestige in string
37 is ok
38 is ok
not found  No
not found  Handicap
not found  Permit
Henrys in string
Authorized in string
Prestige in string
Henrys in string


Prestige in string
Bigtow in string
345 is ok
346 is ok
Authorized in string
not found  Towing
not found  Commerical
not found  Veh
Speedy in string
Henrys in string
not found  ()
350 is ok
not found  Reason
not found  A
not found  &
not found  M
not found  Towing
Henrys in string
353 is ok
not found  Disabled
not found  Veh
Henrys in string
not found  ()
Auth in string
Authorized in string
Speedy in string
not found  On
not found  Property
358 is ok
Am in string
Fire in string
not found  Ln
360 is ok
Henrys in string
362 is ok
Henrys in string
364 is ok
not found  A
Prestige in string
Authorized in string
not found  Towing
Prestige in string
368 is ok
369 is ok
370 is ok
Henrys in string
Henrys in string
Bigtow in string
Henrys in string
375 is ok
Henrys in string
377 is ok
378 is ok
Henrys in string
Authorized in string
Authroized in string
382 is ok
Prestige in string
not found  Towing
not found  To
not found  For
384 is ok
not found  Authrized
Drive in string
Henrys in string
388 i

Rbf in string
not found  Towing
not found  To
not found  For
Prestige in string
not found  Reserved
not found  Space
Prestige in string
863 is ok
Henrys in string
Big in string
not found  Tow
Henrys in string
867 is ok
868 is ok
869 is ok
Henrys in string
Authorized in string
not found  Towing
Custom in string
not found  Towing
not found  To
not found  For
Henrys in string
not found  ***
not found  Unable
not found  To
not found  Provide
not found  ****
not found  Exp
Henrys in string
875 is ok
Authorized in string
not found  Pkd
not found  In
not found  Reserved
not found  Spot
Henrys in string
Henrys in string
Authorized in string
not found  Exp
not found  Perm
Big in string
not found  Tow
not found  Divrsfd
882 is ok
not found  ***
not found  Unable
not found  To
not found  Provide
not found  ***
Henrys in string
884 is ok
Led in string
not found  Towing
886 is ok
not found  Visitor
Henrys in string
888 is ok
not found  Pvt
not found  Prop
Prestige in string
890 is ok
Henrys in stri

Prestige in string
Authorized in string
Henrys in string
1364 is ok
Auth in string
not found  Hanidcap
not found  Space
Henrys in string
Authorized in string
not found  Abnd
Millers in string
not found  Towing
not found  And
not found  Recovery
Authorized in string
not found  T
not found  Cl
Henrys in string
1371 is ok
Henrys in string
not found  Property
1373 is ok
not found  No
not found  Visitor
not found  Placard
Henrys in string
Authorized in string
not found  Owner
not found  Notif
Henrys in string
not found  Henry
not found  Call
not found  In
not found  WalkOff
Auburn in string
Henrys in string
1380 is ok
Henrys in string
1382 is ok
1383 is ok
not found  In
Henrys in string
Authorized in string
Led in string
not found  Towing
not found  ()
1387 is ok
1388 is ok
not found  Walkoff
Millers in string
Authorized in string
not found  Owner
not found  Notified
1391 is ok
1392 is ok
1393 is ok
not found  Reserved
Big in string
not found  Tow
not found  Expired
not found  Tags
RickiesT

Authorized in string
Henrys in string
1966 is ok
not found  Exp
not found  Tag
Rickies in string
Henrys in string
Henrys in string
A&M in string
not found  Towing
Henrys in string
1972 is ok
not found  Prop
Prestige in string
Authorized in string
Custom in string
not found  A
not found  &
not found  M
1977 is ok
not found  Credit
not found  Accept
not found  Latenite
1979 is ok
not found  Private
not found  Property
not found  Pretigue
Henrys in string
RickiesTowing in string
Prestige in string
1984 is ok
Authorized in string
not found  Towing
not found  Is
Led in string
1987 is ok
1988 is ok
Diversified in string
Henrys in string
1991 is ok
Henrys in string
1993 is ok
not found  Pvp
Prestige in string
not found  Reason
Prestige in string
Authorize in string
not found  The
not found  Tag
not found  Comes
not found  Back
not found  To
not found  A
not found  Honda
not found  Accrd
not found  With
not found  The
not found  HgcgA
1997 is ok
Custom in string
Henrys in string
Prestige in st

...

Speedy in string
not found  Cmax
not found  S
not found  Private
not found  Property
Prestige in string
Big in string
not found  Tow
17267 is ok
Prestige in string
Rbf in string
not found  Towing
not found  ()
Drive in string
not found  Towing
17271 is ok
not found  Va
not found  Temp
not found  Tag
Henrys in string
Speedy in string
Auburn in string
not found  Rickiess
not found  No
not found  Visiots
Aubrun in string
not found  Toyt
not found  Financial
Henrys in string
not found  Wells
not found  Fargo
Etc in string
not found  Recovery
not found  Reason
Henrys in string
Drive in string
not found  Handicap
not found  W
not found  No
not found  Placard
17282 is ok
Henrys in string
17284 is ok
not found  Ifo
not found  Dumpster
Diversified in string
not found  Rickiess
not found  Back
not found  To
not found  Lot
Henrys in string
not found  No
not found  P
not found  Parked
not found  Over
not found  Time
not found  Limit
Rickies in string
17289 is ok
17290 is ok
Authorized in string
17

Henrys in string
not found  And
not found  Flat
not found  Tire
not found  Reason
Henrys in string
17742 is ok
Rickies in string
17744 is ok
Henrys in string
17746 is ok
17747 is ok
not found  Reserved
Millers in string
17749 is ok
not found  Authozed
Henrys in string
not found  Reason
Henrys in string
Henrys in string
not found  Dbl
not found  Pkd
not found  And
17754 is ok
Henrys in string
17756 is ok
Drive in string
17758 is ok
Prestige in string
Prestige in string
Auburn in string
not found  Towing
17762 is ok
17763 is ok
Authorized in string
Henrys in string
not found  Dbl
17766 is ok
not found  On
not found  The
not found  Run
not found  Towing
not found  ()
17768 is ok
not found  Community
not found  Loans
not found  Of
not found  America
17770 is ok
17771 is ok
Henrys in string
17773 is ok
Henrys in string
not found  Private
not found  Property
Prestige in string
17776 is ok
17777 is ok
Henrys in string
Henrys in string
not found  Towing
not found  To
not found  For
not found  

Henrys in string
not found  Towing
Speedys in string
Grace in string
not found  Towing
18047 is ok
Authorized in string
Authorized in string
18050 is ok
Authorized in string
Big in string
not found  Tow
Henrys in string
not found  Visitori
not found  In
not found  Resident
not found  Space
Henrys in string
Henrys in string
Auburn in string
not found  Towing
Authorized in string
Authorized in string
18059 is ok
Drive in string
18061 is ok
Henrys in string
Prestige in string
not found  ()
18064 is ok
18065 is ok
not found  Finance
not found  Is
Santander in string
Henrys in string
18068 is ok
Henrys in string
not found  Firelane
Henrys in string
Authorized in string
Fire in string
not found  Lane
18072 is ok
18073 is ok
Rickies in string
Henrys in string
18076 is ok
Henrys in string
not found  Two
not found  Spaces
Led in string
Henrys in string
18080 is ok
Authorized in string
18082 is ok
not found  Exp
not found  Visitor
not found  Pass
Authorized in string
18084 is ok
Henrys in string

Greenwood in string
not found  Recovery
Drive in string
not found  Towing
Fire in string
not found  Lane
Diversified in string
Prestige in string
18578 is ok
18579 is ok
Authorized in string
Henrys in string
18582 is ok
not found  First
not found  Help
not found  Financial
18584 is ok
Drive in string
not found  Towing
Henrys in string
not found  A
Prestige in string
not found  Towing
18588 is ok
not found  Exp
not found  Tag
not found  Aband
Speedys in string
Fire in string
not found  Ln
Henrys in string
18591 is ok
Henrys in string
not found  Silver
not found  Spring
not found  Towing
Henrys in string
not found  No
not found  P
not found  Westlake
not found  Financial
Cti in string
not found  Towing
Auburn in string
18597 is ok
not found  Prop
Prestige in string
Henrys in string
Prestige in string
not found  Towing
not found  To
not found  Prv
Big in string
not found  Tow
not found  No
not found  Permit
not found  Capital
not found  One
Diversified in string
not found  Reason
Henrys i

NameError: name 'tow' is not defined

In [13]:
#Place State = MD in column 
towing["state"] = "MD"

#Split geo-point to retrieve latitude and lomgitude
towing_geo = towing["geo_location"].str.split(" ", n = 5, expand = True)
print(towing_geo)

#create new towing dataset columns from split colums columns: "latitude", "longitude", and 
#"location" for use in FCC census block lookup
towing["longitude"] = towing_geo.iloc[:,1]
towing["longitude"] = towing["longitude"].str.replace("(", "")
towing["latitude"] = towing_geo.iloc[:,2]
towing["latitude"] = towing["latitude"].str.replace(")", "")
towing["location"] = towing["latitude"] + "," + towing["longitude"]
towing.head()

           0            1           2
0      POINT  (-76.998141  38.980389)
1      POINT  (-77.261484  39.179888)
2      POINT  (-77.042077  38.999696)
3      POINT  (-76.951448  39.083196)
4      POINT  (-77.204414  39.163187)
...      ...          ...         ...
18859  POINT  (-76.939653  39.072248)
18860  POINT   (-77.06692  39.088854)
18861  POINT   (-77.06692  39.088854)
18862  POINT  (-76.999533  38.998597)
18863  POINT  (-77.181901  39.191003)

[18864 rows x 3 columns]


,tow_date,tow_reason,storage_company,notes,incident_date,street,city,geo_location,state,longitude,latitude,location
0,2020-05-18,PKG,Henrys,Henrys,05/20/0020,7411 Aspen Ct,Takoma Park,POINT (-76.998141 38.980389),MD,-76.998141,38.980389,"38.980389,-76.998141"
1,2020-04-13,NOP,Authorized,Authorized,04/15/0020,19757 Crystal Rock Dr,Germantown,POINT (-77.261484 39.179888),MD,-77.261484,39.179888,"39.179888,-77.261484"
2,2020-04-09,PKG,Prestige,Handicap Spot Without Placard Or Tag Presti...,09/22/0004,1970 Rosemary Hills Dr,Silver Spring,POINT (-77.042077 38.999696),MD,-77.042077,38.999696,"38.999696,-77.042077"
3,2020-04-25,PKG,Henrys,Henrys,04/27/0020,2800 Shepperton Ter,Silver Spring,POINT (-76.951448 39.083196),MD,-76.951448,39.083196,"39.083196,-76.951448"
4,2020-04-30,NOP,Henrys,Henrys,05/02/0020,18710 Walkers Choice Rd,Montgomery Village,POINT (-77.204414 39.163187),MD,-77.204414,39.163187,"39.163187,-77.204414"


In [15]:
len(towing)

18864

In [16]:
#Write csv files to local drive for processing FCC Block extraction
max_row = len("towing") - 1
batch1 = towing[0:9999]
batch2 = towing[10000:19999]
batch1.to_csv('/Users/elizabethclick/Desktop/DATA205/TowBatch1.csv', index = False)
batch2.to_csv('/Users/elizabethclick/Desktop/DATA205/TowBatch2.csv', index = False)
len(batch2)

8864

In [17]:
#process in short bursts
process_chunksize = 10
#Initialize separated dataframe "towing" saved on drive 
os.chdir('/Users/elizabethclick/Desktop/DATA205')
mainCSV1 = 'TowBatch1.csv'
mainCSV2 = 'TowBatch2.csv'

mainDF1 = pd.read_csv(mainCSV1, chunksize=process_chunksize)
mainDF2 = pd.read_csv(mainCSV2, chunksize=process_chunksize)

In [18]:
#Pull desired census data from Census API
census_api_url='https://api.census.gov/data/2019/acs/acs5?key=5533410471151873cdf3e32fd6cacb45753daa2a&get=NAME,B29004_001E,B01003_001E,B02001_003E&for=block%20group:*&in=state:24&in=county:031&in=tract:*'
responseCensus = requests.get(census_api_url, headers={'Content-Type': 'application/json'})
textCensus=json.loads(responseCensus.content)
tractDF = pd.DataFrame(textCensus)
print('Shape of census tract file: ' + str(tractDF.shape))

tractDF.rename(columns = tractDF.iloc[0], inplace=True)
tractDF = tractDF.iloc[1:]
tractDF.rename(index=str,columns={'B29004_001E':'hh_income','B01003_001E':'pop_total',
                                  'B02001_003E':'pop_black'}, inplace=True)

Shape of census tract file: (615, 8)


In [19]:
tractDF.head()

,NAME,hh_income,pop_total,pop_black,state,county,tract,block group
1,"Block Group 4, Census Tract 7032.08, Montgomer...",158125,1646,319,24,031,703208,4
2,"Block Group 4, Census Tract 7015.09, Montgomer...",56786,3041,1284,24,031,701509,4
3,"Block Group 2, Census Tract 7015.08, Montgomer...",68750,590,263,24,031,701508,2
4,"Block Group 1, Census Tract 7039.01, Montgomer...",160938,1553,354,24,031,703901,1
5,"Block Group 2, Census Tract 7057.02, Montgomer...",134000,1697,21,24,031,705702,2


In [20]:
tract2 = tractDF.sort_values(by='tract')
tract2.head()

,NAME,hh_income,pop_total,pop_black,state,county,tract,block group
97,"Block Group 1, Census Tract 7001.01, Montgomer...",154762,1922,184,24,031,700101,1
158,"Block Group 3, Census Tract 7001.01, Montgomer...",135208,800,10,24,031,700101,3
159,"Block Group 2, Census Tract 7001.01, Montgomer...",208929,1931,39,24,031,700101,2
494,"Block Group 3, Census Tract 7001.03, Montgomer...",172778,3112,345,24,031,700103,3
176,"Block Group 2, Census Tract 7001.03, Montgomer...",218173,1017,193,24,031,700103,2


In [21]:
tractDF['tract'][0]
tractDF.dtypes

NAME           object
hh_income      object
pop_total      object
pop_black      object
state          object
county         object
tract          object
block group    object
dtype: object

In [22]:
#concatenate tract and block codes
tractDF['block group'] = tractDF['block group'].apply(str)
tractDF['tract'] = tractDF['tract'].apply(str)

for i in range(0,len(tractDF)):
    tractDF['block group'][i] = tractDF['tract'][i] + tractDF['block group'][i]
    
print(tractDF.head())


                                                NAME hh_income pop_total  \
1  Block Group 4, Census Tract 7032.08, Montgomer...    158125      1646   
2  Block Group 4, Census Tract 7015.09, Montgomer...     56786      3041   
3  Block Group 2, Census Tract 7015.08, Montgomer...     68750       590   
4  Block Group 1, Census Tract 7039.01, Montgomer...    160938      1553   
5  Block Group 2, Census Tract 7057.02, Montgomer...    134000      1697   

  pop_black state county   tract block group  
1       319    24    031  703208     7032084  
2      1284    24    031  701509     7015094  
3       263    24    031  701508     7015082  
4       354    24    031  703901     7039011  
5        21    24    031  705702     7057022  


In [23]:
for column in tractDF.columns:
    print(column)
tractDF.dtypes

NAME
hh_income
pop_total
pop_black
state
county
tract
block group


NAME           object
hh_income      object
pop_total      object
pop_black      object
state          object
county         object
tract          object
block group    object
dtype: object

In [24]:
#Define tract finding function using FCC API with longitude and latitudes used to add block codes to towing dataset
def getblock(lonlat):
    block = -999
    api_url='https://geo.fcc.gov/api/census/area?lat='+str(lonlat[0])+'&lon='+str(lonlat[1])+'&format=json'
    headers = {'Content-Type': 'application/json'}
    response = requests.get(api_url, headers=headers)
    try:
        block=json.loads(response.content.decode('utf-8'))['results'][0]['block_fips'][5:12]
    except:
        print('No block found from FCC geo API')
    return block

In [25]:
#process first 10000 rows of towing dataframe - get block codes in a new column
df1 = pd.DataFrame()
itld=0
itlu=1
    
for mainDF_chunk in mainDF1:
            nLinesMainCSVd=itld*process_chunksize
            nLinesMainCSVu=itlu*process_chunksize
            print('New group from main DF, lines '+str(nLinesMainCSVd) + ' to ' + str(nLinesMainCSVu))

            mainDF_chunk['location']=mainDF_chunk['location'].map(lambda x: eval(str(x)))
            print('Dimension of main DF group:' + str(mainDF_chunk.shape))
    
            # Add block from FCC geo to towing dataset in new column "block"
            mainDF_chunk['block']=mainDF_chunk.location.map(lambda x: getblock(x))
            print(mainDF_chunk["block"])
           
            df1 = df1.append(mainDF_chunk)
    
            itlu+=1
            itld+=1

New group from main DF, lines 0 to 10
Dimension of main DF group:(10, 12)
0    7017041
1    7008182
2    7026022
3    7014141
4    7007212
5    7007131
6    7007233
7    7008153
8    7007212
9    7008193
Name: block, dtype: object
New group from main DF, lines 10 to 20
Dimension of main DF group:(10, 12)
10    7008161
11    7009041
12    7007151
13    7008111
14    7007232
15    7007111
16    7014222
17    7026022
18    7025001
19    7008131
Name: block, dtype: object
New group from main DF, lines 20 to 30
Dimension of main DF group:(10, 12)
20    7008134
21    7006141
22    7032141
23    7014221
24    7015051
25    7048033
26    7014222
27    7008172
28    7032132
29    7008193
Name: block, dtype: object
New group from main DF, lines 30 to 40
Dimension of main DF group:(10, 12)
30    7008113
31    7014172
32    7015051
33    7048032
34    7032141
35    7014221
36    7007233
37    7016011
38    7008162
39    7003062
Name: block, dtype: object
New group from main DF, lines 40 to 50
Dime

330    7044012
331    7032091
332    7006141
333    7025004
334    7007101
335    7032091
336    7060123
337    7007212
338    7008172
339    7016011
Name: block, dtype: object
New group from main DF, lines 340 to 350
Dimension of main DF group:(10, 12)
340    7003083
341    7008121
342    7032022
343    7008113
344    7008154
345    7009041
346    7007104
347    7023021
348    7007101
349    7003102
Name: block, dtype: object
New group from main DF, lines 350 to 360
Dimension of main DF group:(10, 12)
350    7007101
351    7007191
352    7007202
353    7007241
354    7026012
355    7015051
356    7032141
357    7006141
358    7032132
359    7008152
Name: block, dtype: object
New group from main DF, lines 360 to 370
Dimension of main DF group:(10, 12)
360    7007221
361    7003044
362    7032091
363    7007212
364    7007042
365    7007061
366    7007212
367    7032133
368    7007212
369    7008182
Name: block, dtype: object
New group from main DF, lines 370 to 380
Dimension of main DF

660    7014174
661    7008172
662    7007172
663    7023023
664    7010041
665    7003083
666    7032162
667    7003091
668    7015051
669    7032162
Name: block, dtype: object
New group from main DF, lines 670 to 680
Dimension of main DF group:(10, 12)
670    7008102
671    7032141
672    7014222
673    7008134
674    7007191
675    7007191
676    7006131
677    7014174
678    7007191
679    7007232
Name: block, dtype: object
New group from main DF, lines 680 to 690
Dimension of main DF group:(10, 12)
680    7008172
681    7008121
682    7012182
683    7015051
684    7008231
685    7015051
686    7012111
687    7003102
688    7014222
689    7008133
Name: block, dtype: object
New group from main DF, lines 690 to 700
Dimension of main DF group:(10, 12)
690    7008102
691    7008162
692    7011012
693    7007202
694    7014101
695    7032141
696    7015092
697    7015092
698    7016022
699    7012182
Name: block, dtype: object
New group from main DF, lines 700 to 710
Dimension of main DF

990    7014222
991    7015092
992    7014221
993    7038001
994    7032133
995    7017033
996    7023023
997    7060121
998    7014232
999    7032141
Name: block, dtype: object
New group from main DF, lines 1000 to 1010
Dimension of main DF group:(10, 12)
1000    7008321
1001    7012182
1002    7014102
1003    7007101
1004    7008134
1005    7014232
1006    7001041
1007    7036011
1008    7014221
1009    7032141
Name: block, dtype: object
New group from main DF, lines 1010 to 1020
Dimension of main DF group:(10, 12)
1010    7008172
1011    7032133
1012    7023021
1013    7048033
1014    7015051
1015    7018001
1016    7012182
1017    7007101
1018    7015092
1019    7032141
Name: block, dtype: object
New group from main DF, lines 1020 to 1030
Dimension of main DF group:(10, 12)
1020    7007233
1021    7007101
1022    7032071
1023    7026012
1024    8073052
1025    7038001
1026    7007113
1027    8073052
1028    7015052
1029    7014103
Name: block, dtype: object
New group from main DF, l

1300    7012212
1301    7009041
1302    7008342
1303    7013123
1304    7016022
1305    7009041
1306    7014102
1307    7009041
1308    7038001
1309    7032132
Name: block, dtype: object
New group from main DF, lines 1310 to 1320
Dimension of main DF group:(10, 12)
1310    7008103
1311    7003101
1312    7032132
1313    7023012
1314    7009042
1315    7008134
1316    7008221
1317    7026012
1318    7014172
1319    7025004
Name: block, dtype: object
New group from main DF, lines 1320 to 1330
Dimension of main DF group:(10, 12)
1320    7032092
1321    7012011
1322    7038001
1323    7060123
1324    7013123
1325    7008221
1326    7008181
1327    7025001
1328    7038001
1329    7038001
Name: block, dtype: object
New group from main DF, lines 1330 to 1340
Dimension of main DF group:(10, 12)
1330    7013131
1331    7007112
1332    7023023
1333    7038001
1334    7014102
1335    7025004
1336    7008184
1337    7012212
1338    7001053
1339    7007061
Name: block, dtype: object
New group from 

...

8740    7008184
8741    7038001
8742    7032162
8743    7032131
8744    7016011
8745    7048062
8746    7007132
8747    7008221
8748    7032202
8749    7015051
Name: block, dtype: object
New group from main DF, lines 8750 to 8760
Dimension of main DF group:(10, 12)
8750    7008321
8751    7007061
8752    7032071
8753    7028002
8754    7007131
8755    7007231
8756    7014222
8757    7012152
8758    7008182
8759    7003041
Name: block, dtype: object
New group from main DF, lines 8760 to 8770
Dimension of main DF group:(10, 12)
8760    7008103
8761    7019001
8762    7014203
8763    7014211
8764    7038001
8765    7032162
8766    7015051
8767    7007132
8768    7032141
8769    7040003
Name: block, dtype: object
New group from main DF, lines 8770 to 8780
Dimension of main DF group:(10, 12)
8770    7014231
8771    7012141
8772    7008241
8773    7014103
8774    7008103
8775    7028002
8776    7016022
8777    7013151
8778    7012182
8779    7015051
Name: block, dtype: object
New group from 

9050    7002072
9051    7008172
9052    7060123
9053    7014232
9054    7032163
9055    7025001
9056    7017031
9057    7015083
9058    7008154
9059    7008134
Name: block, dtype: object
New group from main DF, lines 9060 to 9070
Dimension of main DF group:(10, 12)
9060    7032132
9061    7008182
9062    7007162
9063    7008221
9064    7038001
9065    7032132
9066    7009041
9067    7014155
9068    7007202
9069    7025004
Name: block, dtype: object
New group from main DF, lines 9070 to 9080
Dimension of main DF group:(10, 12)
9070    7044031
9071    7008172
9072    7014222
9073    7017012
9074    7007233
9075    7007231
9076    7012153
9077    7006141
9078    7024023
9079    7007131
Name: block, dtype: object
New group from main DF, lines 9080 to 9090
Dimension of main DF group:(10, 12)
9080    7019001
9081    7016011
9082    7009031
9083    7032162
9084    7009011
9085    7025001
9086    7032142
9087    7008301
9088    7032124
9089    7032143
Name: block, dtype: object
New group from 

9360    7014211
9361    7012053
9362    7007201
9363    7021014
9364    7032141
9365    7009031
9366    7032201
9367    7021011
9368    7008151
9369    7014103
Name: block, dtype: object
New group from main DF, lines 9370 to 9380
Dimension of main DF group:(10, 12)
9370    7014144
9371    7016022
9372    7008171
9373    7012012
9374    7008111
9375    7007232
9376    7014103
9377    7009041
9378    7014221
9379    7038001
Name: block, dtype: object
New group from main DF, lines 9380 to 9390
Dimension of main DF group:(10, 12)
9380    7025001
9381    7060123
9382    7015083
9383    7003092
9384    7039022
9385    7008172
9386    7014203
9387    7032133
9388    7032122
9389    7032141
Name: block, dtype: object
New group from main DF, lines 9390 to 9400
Dimension of main DF group:(10, 12)
9390    7008182
9391    7015083
9392    7014211
9393    7008171
9394    7008321
9395    7032133
9396    7014173
9397    7008181
9398    7009031
9399    7032022
Name: block, dtype: object
New group from 

9670    7008221
9671    7007131
9672    7013131
9673    7008153
9674    7044031
9675    7007162
9676    7032092
9677    7015083
9678    7014222
9679    7007232
Name: block, dtype: object
New group from main DF, lines 9680 to 9690
Dimension of main DF group:(10, 12)
9680    7007131
9681    7008134
9682    7007133
9683    7013151
9684    7025001
9685    7008172
9686    7014233
9687    7026022
9688    7008173
9689    7014173
Name: block, dtype: object
New group from main DF, lines 9690 to 9700
Dimension of main DF group:(10, 12)
9690    7016022
9691    7003101
9692    7015083
9693    7007132
9694    7009011
9695    7007222
9696    7008321
9697    7006141
9698    7032071
9699    7009041
Name: block, dtype: object
New group from main DF, lines 9700 to 9710
Dimension of main DF group:(10, 12)
9700    7014103
9701    7001051
9702    7014222
9703    7015051
9704    7003111
9705    7012011
9706    7015083
9707    7003044
9708    7007241
9709    7032141
Name: block, dtype: object
New group from 

9980    7007173
9981    7038001
9982    7032141
9983    7008132
9984    7008181
9985    7007201
9986    7008121
9987    7014222
9988    7025001
9989    7008221
Name: block, dtype: object
New group from main DF, lines 9990 to 10000
Dimension of main DF group:(9, 12)
9990    7032123
9991    7007111
9992    7027001
9993    7002082
9994    7021014
9995    7016022
9996    7007132
9997    7032203
9998    7008194
Name: block, dtype: object


In [26]:
#check csv splits to see that tables maintain all rows
df1.to_csv('/Users/elizabethclick/Desktop/DATA205/df1.csv', index = False)
df1.tail()


,tow_date,tow_reason,storage_company,notes,incident_date,street,city,geo_location,state,longitude,latitude,location,block
9994,2020-08-10,NOP,Henrys,Henrys **Number **,10/22/0008,8830 Piney Branch Rd,Silver Spring,POINT (-76.993067 38.999737),MD,-76.993067,38.999737,"(38.999737, -76.993067)",7021014
9995,2020-08-27,NOP,Drive,Drive,08/29/0020,475 Southampton Dr,Silver Spring,POINT (-76.984197 39.0062),MD,-76.984197,39.006200,"(39.0062, -76.984197)",7016022
9996,2020-08-11,FIRE,Authorized,Authorized,11/22/0008,18063 Royal Bonnet Cir,Montgomery Village,POINT (-77.193019 39.153498),MD,-77.193019,39.153498,"(39.153498, -77.193019)",7007132
9997,2020-08-30,EXP,Speedy,Speedy,09/01/0020,2600 Camelback Ln,Silver Spring,POINT (-77.057109 39.105974),MD,-77.057109,39.105974,"(39.105974, -77.057109)",7032203
9998,2020-08-02,PKG,Speedy,Speedy,02/22/0008,19334 Hottinger Cir,Germantown,POINT (-77.253352 39.173518),MD,-77.253352,39.173518,"(39.173518, -77.253352)",7008194


In [27]:
#process second batch (remaining) rows of towing dataframe - get block codes in a new column
df2 = pd.DataFrame()
itld=0
itlu=1
    
for mainDF_chunk in mainDF2:
            nLinesMainCSVd=itld*process_chunksize
            nLinesMainCSVu=itlu*process_chunksize
            print('New group from main DF, lines '+str(nLinesMainCSVd) + ' to ' + str(nLinesMainCSVu))

            
            mainDF_chunk['location']=mainDF_chunk['location'].map(lambda x: eval(str(x)))
            #mainDF_chunk[['tow_date']].apply(pd.to_datetime,errors='coerce')
            print('Dimension of main DF group:' + str(mainDF_chunk.shape))
    
            # Add block from FCC geo to my main dataset in new column "block"
            mainDF_chunk['block']=mainDF_chunk.location.map(lambda x: getblock(x))
            print(mainDF_chunk["block"])
            
            df2 = df2.append(mainDF_chunk)
    
            itlu+=1
            itld+=1

New group from main DF, lines 0 to 10
Dimension of main DF group:(10, 12)
0    7003101
1    7007042
2    7014173
3    7012153
4    7023012
5    7014174
6    7014155
7    7016022
8    7023011
9    7008321
Name: block, dtype: object
New group from main DF, lines 10 to 20
Dimension of main DF group:(10, 12)
10    7048033
11    7020003
12    7009031
13    7024023
14    7008113
15    7024023
16    7007221
17    7008321
18    7012053
19    7040001
Name: block, dtype: object
New group from main DF, lines 20 to 30
Dimension of main DF group:(10, 12)
20    7014103
21    7025001
22    7014211
23    7012181
24    7012181
25    7025001
26    7008132
27    7015083
28    7012191
29    7008321
Name: block, dtype: object
New group from main DF, lines 30 to 40
Dimension of main DF group:(10, 12)
30    7008103
31    7032162
32    7007181
33    7008134
34    7008181
35    7007104
36    7014173
37    7008321
38    7016022
39    7032143
Name: block, dtype: object
New group from main DF, lines 40 to 50
Dime

330    7008171
331    7032092
332    7007232
333    7009031
334    7008102
335    7015051
336    7007241
337    7028002
338    7003063
339    7008201
Name: block, dtype: object
New group from main DF, lines 340 to 350
Dimension of main DF group:(10, 12)
340    7032101
341    7007152
342    7012011
343    7012191
344    7007171
345    7007103
346    7008321
347    7032162
348    7003091
349    7057013
Name: block, dtype: object
New group from main DF, lines 350 to 360
Dimension of main DF group:(10, 12)
350    7006142
351    7016011
352    7012182
353    7019001
354    7026011
355    7007042
356    7008134
357    7008183
358    7014174
359    7014232
Name: block, dtype: object
New group from main DF, lines 360 to 370
Dimension of main DF group:(10, 12)
360    7039022
361    7008321
362    7008111
363    7003082
364    7007193
365    7014174
366    7032123
367    7015092
368    7008321
369    7007241
Name: block, dtype: object
New group from main DF, lines 370 to 380
Dimension of main DF

660    7008173
661    7032162
662    7014092
663    7003083
664    7008182
665    7032143
666    7038001
667    7003101
668    7009031
669    7032133
Name: block, dtype: object
New group from main DF, lines 670 to 680
Dimension of main DF group:(10, 12)
670    7003103
671    7025001
672    7032132
673    7007191
674    7008321
675    7008173
676    7008151
677    7013151
678    7008153
679    7032203
Name: block, dtype: object
New group from main DF, lines 680 to 690
Dimension of main DF group:(10, 12)
680    7017021
681    7019001
682    7008181
683    7008321
684    7003063
685    7013131
686    7032162
687    7038001
688    7014103
689    7032162
Name: block, dtype: object
New group from main DF, lines 690 to 700
Dimension of main DF group:(10, 12)
690    7014203
691    7038001
692    7016022
693    7008201
694    7025003
695    7008181
696    7020003
697    7008221
698    7003101
699    7008193
Name: block, dtype: object
New group from main DF, lines 700 to 710
Dimension of main DF

990    7007191
991    7007101
992    7008221
993    7038001
994    7057013
995    7039022
996    7014155
997    7012182
998    7032091
999    7012011
Name: block, dtype: object
New group from main DF, lines 1000 to 1010
Dimension of main DF group:(10, 12)
1000    7026022
1001    7025001
1002    7008181
1003    7008103
1004    7008221
1005    7016011
1006    7008194
1007    7014172
1008    7012182
1009    7008184
Name: block, dtype: object
New group from main DF, lines 1010 to 1020
Dimension of main DF group:(10, 12)
1010    7008342
1011    7010041
1012    7015083
1013    7001033
1014    7007173
1015    7024023
1016    7009041
1017    7032132
1018    7016022
1019    7007162
Name: block, dtype: object
New group from main DF, lines 1020 to 1030
Dimension of main DF group:(10, 12)
1020    7002072
1021    7025001
1022    7008221
1023    7025001
1024    7021014
1025    7008102
1026    7014103
1027    7012011
1028    7003082
1029    7008154
Name: block, dtype: object
New group from main DF, l

1300    7008121
1301    7032143
1302    7012182
1303    7023023
1304    7008134
1305    7014231
1306    7002072
1307    7027002
1308    7044031
1309    7008321
Name: block, dtype: object
New group from main DF, lines 1310 to 1320
Dimension of main DF group:(10, 12)
1310    7007042
1311    7032071
1312    7014211
1313    7008103
1314    7012182
1315    7032071
1316    7007232
1317    7048033
1318    7007222
1319    7038001
Name: block, dtype: object
New group from main DF, lines 1320 to 1330
Dimension of main DF group:(10, 12)
1320    7025001
1321    7032162
1322    7038001
1323    7007173
1324    7008172
1325    7032203
1326    7008181
1327    7060123
1328    7008133
1329    7008321
Name: block, dtype: object
New group from main DF, lines 1330 to 1340
Dimension of main DF group:(10, 12)
1330    7032123
1331    7032143
1332    7015083
1333    7008201
1334    7028002
1335    7008184
1336    7008113
1337    7015083
1338    7008342
1339    7032131
Name: block, dtype: object
New group from 

...

7500    7032092
7501    7014103
7502    7014211
7503    7006141
7504    7021014
7505    7008181
7506    7048061
7507    7032143
7508    7015082
7509    7020001
Name: block, dtype: object
New group from main DF, lines 7510 to 7520
Dimension of main DF group:(10, 12)
7510    7015053
7511    7009041
7512    7007173
7513    7008134
7514    7016011
7515    7008161
7516    7006132
7517    7026021
7518    7032061
7519    7038001
Name: block, dtype: object
New group from main DF, lines 7520 to 7530
Dimension of main DF group:(10, 12)
7520    7012053
7521    7008181
7522    7032071
7523    7021011
7524    7038001
7525    7008172
7526    7048033
7527    7007222
7528    7007233
7529    7019001
Name: block, dtype: object
New group from main DF, lines 7530 to 7540
Dimension of main DF group:(10, 12)
7530    7007163
7531    7008102
7532    7012182
7533    7016011
7534    7007181
7535    7015051
7536    7003111
7537    7014172
7538    7015083
7539    7003092
Name: block, dtype: object
New group from 

7810    7025004
7811    7008134
7812    7015083
7813    7032141
7814    7032061
7815    7007173
7816    7007231
7817    7015081
7818    7008172
7819    7002072
Name: block, dtype: object
New group from main DF, lines 7820 to 7830
Dimension of main DF group:(10, 12)
7820    7016022
7821    7014231
7822    7012053
7823    7008181
7824    7032071
7825    7014172
7826    7016022
7827    7002071
7828    7008181
7829    7008103
Name: block, dtype: object
New group from main DF, lines 7830 to 7840
Dimension of main DF group:(10, 12)
7830    7014172
7831    7016011
7832    7007211
7833    7003063
7834    7008134
7835    7007222
7836    7015083
7837    7014204
7838    7006141
7839    7014232
Name: block, dtype: object
New group from main DF, lines 7840 to 7850
Dimension of main DF group:(10, 12)
7840    7024023
7841    7016011
7842    7008173
7843    7025001
7844    7008173
7845    7015051
7846    7008134
7847    7012153
7848    7055023
7849    7032132
Name: block, dtype: object
New group from 

8120    7007241
8121    7007133
8122    7019001
8123    7008132
8124    7014232
8125    7002072
8126    7007133
8127    7016011
8128    7008332
8129    7037023
Name: block, dtype: object
New group from main DF, lines 8130 to 8140
Dimension of main DF group:(10, 12)
8130    7007192
8131    7016022
8132    7032133
8133    7032092
8134    7007192
8135    7008153
8136    7019001
8137    7007233
8138    7006082
8139    7013131
Name: block, dtype: object
New group from main DF, lines 8140 to 8150
Dimension of main DF group:(10, 12)
8140    7003103
8141    7007173
8142    7008152
8143    7016011
8144    7038001
8145    7016011
8146    7007111
8147    7014181
8148    7008184
8149    7008172
Name: block, dtype: object
New group from main DF, lines 8150 to 8160
Dimension of main DF group:(10, 12)
8150    7003062
8151    7023023
8152    7015083
8153    7032162
8154    7007131
8155    7060123
8156    7009041
8157    7032122
8158    7007061
8159    7019001
Name: block, dtype: object
New group from 

8430    7020003
8431    7007192
8432    7012053
8433    7008161
8434    7012011
8435    7008162
8436    7007061
8437    7023011
8438    7016022
8439    7008231
Name: block, dtype: object
New group from main DF, lines 8440 to 8450
Dimension of main DF group:(10, 12)
8440    7012113
8441    7039022
8442    7007131
8443    7001042
8444    7008172
8445    7048041
8446    7014181
8447    7028002
8448    7048041
8449    7025001
Name: block, dtype: object
New group from main DF, lines 8450 to 8460
Dimension of main DF group:(10, 12)
8450    7007113
8451    7007043
8452    7038001
8453    7017021
8454    7032061
8455    7012164
8456    7007212
8457    7016011
8458    7008151
8459    7009031
Name: block, dtype: object
New group from main DF, lines 8460 to 8470
Dimension of main DF group:(10, 12)
8460    7013131
8461    7009041
8462    7019001
8463    7014232
8464    7021011
8465    7012153
8466    7010021
8467    7007233
8468    7023011
8469    7032071
Name: block, dtype: object
New group from 

8740    7060121
8741    7014172
8742    7016011
8743    7016022
8744    7007192
8745    7008173
8746    7012112
8747    7006141
8748    7025001
8749    7008134
Name: block, dtype: object
New group from main DF, lines 8750 to 8760
Dimension of main DF group:(10, 12)
8750    7015051
8751    7008184
8752    7015051
8753    7006134
8754    7008102
8755    7009031
8756    7009011
8757    7015093
8758    7007193
8759    7032071
Name: block, dtype: object
New group from main DF, lines 8760 to 8770
Dimension of main DF group:(10, 12)
8760    7003091
8761    7017021
8762    7014232
8763    7006134
8764    7012012
8765    7007131
8766    7016011
8767    7003102
8768    7026012
8769    7008231
Name: block, dtype: object
New group from main DF, lines 8770 to 8780
Dimension of main DF group:(10, 12)
8770    7032162
8771    7008162
8772    7014211
8773    7014222
8774    7007193
8775    7001052
8776    7015051
8777    7007201
8778    7032071
8779    7032083
Name: block, dtype: object
New group from 

In [28]:
df2.to_csv('/Users/elizabethclick/Desktop/DATA205/df2.csv', index = False)
df2.head()


,tow_date,tow_reason,storage_company,notes,incident_date,street,city,geo_location,state,longitude,latitude,location,block
0,2020-08-07,FIRE,Tcl,On The Run (),07/22/0008,13069 Wisteria Dr,Germantown,POINT (-77.272002 39.178015),MD,-77.272002,39.178015,"(39.178015, -77.272002)",7003101
1,2020-08-23,NOP,Big,Big Tow,08/25/0020,15955 Frederick Rd,Derwood,POINT (-77.169576 39.11769),MD,-77.169576,39.117690,"(39.11769, -77.169576)",7007042
2,2020-08-15,NOP,Henrys,Henrys,08/17/0020,13615 Colgate Way,Silver Spring,POINT (-76.93266 39.078886),MD,-76.932660,39.078886,"(39.078886, -76.93266)",7014173
3,2020-08-25,NOP,Authorized,Authorized,08/27/0020,5405 Tuckerman Ln,Rockville,POINT (-77.1081 39.029089),MD,-77.108100,39.029089,"(39.029089, -77.1081)",7012153
4,2020-08-04,PKG,Henrys,Henrys Unable To Read Vin,04/22/0008,8532 Piney Branch Rd,Silver Spring,POINT (-77.001918 38.998373),MD,-77.001918,38.998373,"(38.998373, -77.001918)",7023012


In [29]:
frames = [df1, df2]
tow_concat = pd.concat(frames)
tow_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18863 entries, 0 to 8863
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tow_date         18863 non-null  object 
 1   tow_reason       18863 non-null  object 
 2   storage_company  18863 non-null  object 
 3   notes            18855 non-null  object 
 4   incident_date    18863 non-null  object 
 5   street           18863 non-null  object 
 6   city             18863 non-null  object 
 7   geo_location     18863 non-null  object 
 8   state            18863 non-null  object 
 9   longitude        18863 non-null  float64
 10  latitude         18863 non-null  float64
 11  location         18863 non-null  object 
 12  block            18863 non-null  object 
dtypes: float64(2), object(11)
memory usage: 2.0+ MB


In [30]:
tow = pd.merge(tow_concat, tractDF, how="inner", left_on="block", right_on="block group", left_index=False, right_index=False,
               sort=True, copy=True, indicator=False, validate=None)
print(tow.head())
tow.info()

     tow_date tow_reason     storage_company  \
0  2020-12-11       REPO        Final Notice   
1  2021-02-10       REPO              Thomas   
2  2020-02-15       REPO  Greenwood Recovery   
3  2020-02-04       REPO  Greenwood Recovery   
4  2020-02-08        NOP              Henrys   

                                               notes incident_date  \
0        Financial Comp Gm Finance     Owner Dan ...    11/22/0012   
1                Reason  Santander  Thomas Recovery     10/23/0002   
2                          Chase Auto   Greenwood       02/15/2020   
3    Mountain America Credit Union   Greenwood Re...    02/04/2020   
4                                           Henrys      02/08/2020   

                  street          city                  geo_location state_x  \
0    21704 Stableview Dr  Gaithersburg  POINT (-77.175624 39.211873)      MD   
1          7321 Brink Rd  Gaithersburg  POINT (-77.148754 39.211605)      MD   
2  21400 Laytonsville Rd  Gaithersburg  POINT (-77

In [31]:
#Final cleaning step - drop columns not needed anymore "notes", "incident_date", street" etc.
print(tow.info())
tow.drop('notes', axis=1, inplace=True) 
tow.drop('incident_date', axis=1, inplace=True)
tow.drop('state_x', axis=1, inplace=True)
tow.drop('NAME', axis = 1, inplace=True)
tow.drop('state_y', axis=1, inplace=True)
tow.drop('block group', axis=1, inplace=True)
tow.drop('county', axis=1, inplace=True)
tow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18767 entries, 0 to 18766
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tow_date         18767 non-null  object 
 1   tow_reason       18767 non-null  object 
 2   storage_company  18767 non-null  object 
 3   notes            18759 non-null  object 
 4   incident_date    18767 non-null  object 
 5   street           18767 non-null  object 
 6   city             18767 non-null  object 
 7   geo_location     18767 non-null  object 
 8   state_x          18767 non-null  object 
 9   longitude        18767 non-null  float64
 10  latitude         18767 non-null  float64
 11  location         18767 non-null  object 
 12  block            18767 non-null  object 
 13  NAME             18767 non-null  object 
 14  hh_income        18767 non-null  object 
 15  pop_total        18767 non-null  object 
 16  pop_black        18767 non-null  object 
 17  state_y     

In [32]:
tow.to_csv('/Users/elizabethclick/Desktop/DATA205/tow_clean.csv', index = False)
tow.head()

,tow_date,tow_reason,storage_company,street,city,geo_location,longitude,latitude,location,block,hh_income,pop_total,pop_black,tract
0,2020-12-11,REPO,Final Notice,21704 Stableview Dr,Gaithersburg,POINT (-77.175624 39.211873),-77.175624,39.211873,"(39.211873, -77.175624)",7001012,208929,1931,39,700101
1,2021-02-10,REPO,Thomas,7321 Brink Rd,Gaithersburg,POINT (-77.148754 39.211605),-77.148754,39.211605,"(39.211605, -77.148754)",7001013,135208,800,10,700101
2,2020-02-15,REPO,Greenwood Recovery,21400 Laytonsville Rd,Gaithersburg,POINT (-77.141329 39.207148),-77.141329,39.207148,"(39.207148, -77.141329)",7001032,218173,1017,193,700103
3,2020-02-04,REPO,Greenwood Recovery,7571 Lindbergh Dr,Gaithersburg,POINT (-77.153158 39.166038),-77.153158,39.166038,"(39.166038, -77.153158)",7001033,172778,3112,345,700103
4,2020-02-08,NOP,Henrys,17604 Phelps Hill Ln,Derwood,POINT (-77.145388 39.147122),-77.145388,39.147122,"(39.147122, -77.145388)",7001033,172778,3112,345,700103


In [33]:
tow = pd.read_csv('/Users/elizabethclick/Desktop/DATA205/tow_clean.csv')
tow.head()

,tow_date,tow_reason,storage_company,street,city,geo_location,longitude,latitude,location,block,hh_income,pop_total,pop_black,tract
0,2020-12-11,REPO,Final Notice,21704 Stableview Dr,Gaithersburg,POINT (-77.175624 39.211873),-77.175624,39.211873,"(39.211873, -77.175624)",7001012,208929,1931,39,700101
1,2021-02-10,REPO,Thomas,7321 Brink Rd,Gaithersburg,POINT (-77.148754 39.211605),-77.148754,39.211605,"(39.211605, -77.148754)",7001013,135208,800,10,700101
2,2020-02-15,REPO,Greenwood Recovery,21400 Laytonsville Rd,Gaithersburg,POINT (-77.141329 39.207148),-77.141329,39.207148,"(39.207148, -77.141329)",7001032,218173,1017,193,700103
3,2020-02-04,REPO,Greenwood Recovery,7571 Lindbergh Dr,Gaithersburg,POINT (-77.153158 39.166038),-77.153158,39.166038,"(39.166038, -77.153158)",7001033,172778,3112,345,700103
4,2020-02-08,NOP,Henrys,17604 Phelps Hill Ln,Derwood,POINT (-77.145388 39.147122),-77.145388,39.147122,"(39.147122, -77.145388)",7001033,172778,3112,345,700103


In [34]:
#CONNECT last chunk to CAPSTONE ANALYSIS and VISUALIZATION